In [171]:
model_ep1 = 'category_kogpt2_trial1_epoch1_loss0.4359'

model_ep3 = 'category_kogpt2_trial1_epoch3_loss0.1805'
model_ep4 = 'category_kogpt2_trial1_epoch4_loss0.1568'
model_ep5 = 'category_kogpt2_trial1_epoch5_loss0.1444'
model_ep6 = 'category_kogpt2_trial1_epoch6_loss0.1366'

model_original = 'kogpt2_80000_5epoch_0.18720loss'

In [124]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -U sentence-transformers

In [125]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
import random
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [127]:
test = pd.read_csv(data_path + 'test.csv')
test.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [172]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# model_name = 'category_kogpt2_trial1_epoch1_loss0.4359'
loaded_model_ep1 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_ep1}').to(device)

loaded_model_ep3 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_ep3}').to(device)
loaded_model_ep4 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_ep4}').to(device)

loaded_model_ep5 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_ep5}').to(device)
loaded_model_ep6 = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_ep6}').to(device)

loaded_model_original = AutoModelForCausalLM.from_pretrained(f'{data_path}models/{model_original}').to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [129]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.95): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [132]:
text = '실내장식이란?'

text = '<q>' + text + '</s><a>'
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = loaded_model_ep6.generate(input_ids,
                        max_length=256,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)


<q>실내장식이란?</s> <a>실내에서 식물을 키우고자 한다면, 적합한 조명을 갖춘 장소에서 자연조명이 있는 곳에 맞는 식물, 공기 정화식물, 그리고 효과적인 관리를 위한 식물을 고려하세요. 

인테리어에 자연적인 요소를 더해보려한다면, 나무 가구, 식물이 뛰어난재, 간편한 관리가 가능한록,000원에서부터 제품을 선택하고, 실내 공기를 적절히 순환시키는 것이 중요합니다. 또한, 식물의 빛을 활용하고 공간을 넓게 보이도록 연출하는 것도 좋은 방법입니다. 예를 들어, 큰 창문이나 투광이 용이한 장소를 만들어주고, 환기를 통해 난방 효율을 높일 수 있습니다. 추가적으로 내부와 외부 환경을 고려하여 LED 천장 장식품을 선택할 수도 있습니다.

이러한 다양한 요소들을 조화롭게 결합하여 공간에 생기와 활력을 불어넣는 것을 권장하고 인테리어를 디자인해주세요.

게다가, 자연의 도움을 받아 새로운 분위기를 조성하기 위해 천연 소재로 만든 커튼과 카펫 및 소품, 인테리어 소품들을 함께 조합하여 사용하는도 고려해볼만 합니다. 이러한 모든 요소들이지만, 공간의 활용도를 높이면서도 편안함을 고려한 디자인을 선택하는 것은 매우 중요한 과정이라고 할 것입니다.</s>


In [133]:
import re

def Chatbot(input_text, model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    text_t = input_text.strip()
    print('question', text_t)
    text = '<q>' + text_t + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        generated_text = generated_text.replace('\n', '')
        generated_text = re.sub(r'\.[^.]*$', '.', generated_text)

        similarity_score = evaluate_similarity(text_t, generated_text)
        generated_texts.append((similarity_score,generated_text))
        print(generated_texts[i]) # 실제 돌릴땐 off

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    print(best_generated_text)


    return best_generated_text

In [134]:
tex = '방청페인트의 종류는\n\n 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입'
tex.replace('\n', '')

'방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다. 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입'

In [135]:
max_len = 200

Chatbot(test['질문'].tolist()[0].strip(), loaded_model_ep6, tokenizer, max_len, device)

question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염

'방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.'

In [136]:
max_len = 200

Chatbot(test['질문'].tolist()[0].strip(), loaded_model_ep6, tokenizer, max_len, device)

question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염

'방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.'

In [137]:
max_len = 200

Chatbot(test['질문'].tolist()[0].strip(), loaded_model_ep5, tokenizer, max_len, device)

question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염

'방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.'

In [138]:
max_len = 200

Chatbot(test['질문'].tolist()[1].strip(), loaded_model_ep6, tokenizer, max_len, device)

question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
(0.6595130089894834, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 이때는 누수 부분을 보수하고, 곰팡이가 발생한 도배지의 부분 또는 전체를 교체하는 것이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.')
(0.6701697618119857, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 이때는 누수 부분을 보수하고, 곰팡이가 발생한 도배지의 부분 또는 전체를 교체하는 것이 필요합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.')
(0.6766288632154465, '도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법

'도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이런 현상이 발생하는 이유, 누구의 책임이며 해결 방법에 대해 설명드리겠습니다.1. 습도가 높은 경우에는 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는 제습기를 사용하거나 적절한 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다.2. 도배지가 계속 젖어 있는 경우에는 곰팡이와 녹이 생길 수 있습니다. 이 문제의 책임은 건물 소유자나 거주자에게 있습니다. 이를 해결하기 위해서는 누수를 해결하고 곰팡이와 녹이 생긴 부분 뿐만 아니라 전체 도배지를 교체하는 복원 작업이 필요합니다. 전문가의 도움을 받는 것이 좋습니다.'

In [139]:
max_len = 200

Chatbot(test['질문'].tolist()[1].strip(), loaded_model_ep5, tokenizer, max_len, device)

question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
(0.6737549748322735, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 누수하는 것을 방지하기 위해 예방할 수 있습니다. 또한 도배지 주변의 누수를 조사하고 보수하는 것도 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이 발생할 수 있습니다. 따라서 누수공사 시에는 이러한 경우에는 먼저 누수 원인을 찾고 해결해야 합니다.')
(0.6485562753677367, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 녹을 수 있으며, 이로 인해 녹이 도배지에 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지에 녹이 베어나올 수 있습니다. 습기를 관리하기 위해서는 제습기나 환기를 통해 실내 습도를 유지하는 것이 중요하며, 또한 누수를 방지하는 등의 조치를 취할 수 있습니다.    1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속이 녹을 수 있습니다. - 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. - 해결 방법: 제습기 가동, 환기를 통해 실내 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것이 좋습니다.')
(0.7006366227354321, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 의미합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습

'도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 의미합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습하거나 누수에 의해 도배지 안쪽의 금속이 녹을 수 있으며, 이로 인해 녹이 도배지에 베어나올 수 있습니다. 따라서 도배지에 녹이 묻어나오는 경우에는 녹오염 뿐만 아니라 곰팡이 발생할 가능성이 있습니다. 따라서 도배지에 녹이 묻어나오는 경우에는 습도 조절과 누수 관리가 중요하며 도배지가 젖은 부분은 신속히 건조시켜야 합니다.'

In [140]:
max_len = 200

Chatbot(test['질문'].tolist()[87].strip(), loaded_model_ep6, tokenizer, max_len, device)

question 제진구조는 건물 구조 중에서 어떤 역할을 하는 건가요? 그리고 중목구조 방식에 대해 좀 더 자세히 설명해 주실 수 있을까요?
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는

'제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.'

In [141]:
test['질문'].tolist()[87]

'제진구조는 건물 구조 중에서 어떤 역할을 하는 건가요? 그리고 중목구조 방식에 대해 좀 더 자세히 설명해 주실 수 있을까요?'

In [142]:
max_len = 200

Chatbot(test['질문'].tolist()[87].strip(), loaded_model_ep5, tokenizer, max_len, device)

question 제진구조는 건물 구조 중에서 어떤 역할을 하는 건가요? 그리고 중목구조 방식에 대해 좀 더 자세히 설명해 주실 수 있을까요?
(0.6129264905767621, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조 방식은 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6129264905767621, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조 방식은 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6129264905767621, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조 방식은 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6129264905767621, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조 방식은 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.')
(0.6164509869971365, '제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재

'제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데, 이 지진 에너지를 감소시키는 장치를 이용하여 건축물의 피해를 줄이는 방법을 의미합니다. 그리고 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 건축한다는 것이 차이점입니다.'

In [143]:
max_len = 200

Chatbot(test['질문'].tolist()[50].strip(), loaded_model_ep5, tokenizer, max_len, device)

question 높은 습도로 인해 몰딩수정이 발생하는 경우가 있을까요? 또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.


'고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.'

In [144]:
max_len = 200

Chatbot(test['질문'].tolist()[50].strip(), loaded_model_ep1, tokenizer, max_len, device)

question 높은 습도로 인해 몰딩수정이 발생하는 경우가 있을까요? 또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.


'고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.'

In [145]:
max_len = 400

Chatbot(test['질문'].tolist()[50].strip(), loaded_model_ep1, tokenizer, max_len, device)

question 높은 습도로 인해 몰딩수정이 발생하는 경우가 있을까요? 또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.


'고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.'

In [146]:
max_len = 200

Chatbot(test['질문'].tolist()[50].strip(), loaded_model_ep6, tokenizer, max_len, device)

question 높은 습도로 인해 몰딩수정이 발생하는 경우가 있을까요? 또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?
(0.6924065919483409, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부를 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.5610581578352513, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외단열 시공은 외단열 시공에 비해 시공 난이도가 낮으며 상대적으로 공사비, 재료비, 인건비가 적게 든다는 장점이 있습니다.')
(0.6764359222518073, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외장재 사이에 있는 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6924065919483409, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부를 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6764359222518073, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외장재 사이에 있는 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부를 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.


'고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부를 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.'

In [147]:
max_len = 400

Chatbot(test['질문'].tolist()[50].strip(), loaded_model_ep5, tokenizer, max_len, device)

question 높은 습도로 인해 몰딩수정이 발생하는 경우가 있을까요? 또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?
(0.6949735416607423, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
(0.6937039138639675, '고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부의 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.')
고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.


'고습도 환경에서 몰딩 소재가 팽창하거나 수축하면서 변형될 수 있습니다. 또한, 내부와 외부 온도차가 클 경우 도배지에 결로가 발생해 곰팡이가 발생할 수 있습니다.'

In [148]:
max_len = 200

Chatbot('인테리어 잘하고 싶은데 팁좀', loaded_model_ep6, tokenizer, max_len, device)

question 인테리어 잘하고 싶은데 팁좀
(0.5163911998271942, '인테리어에서 수납 공간을 늘리는 방법은 다양합니다. 예를 들어, 가구를 활용한 수납, 벽면을 활용한 수납장 설치, 다용도 가구의 활용, 수납함과 서랍을 활용하여 숨겨진 수납 공간을 만들거나 옷장 안에 추가 선반을 설치하는 등의 방법을 통해 효과적인 수납 공간을 만들 수 있습니다.')
(0.5476495176553726, '인테리어에서 색상 선택 시 고려해야 할 몇 가지 팁이 있습니다. 먼저, 색상을 선택할 때는 공간의 크기, 채광 상태, 가구의 색상, 그리고 사용자의 취향과 성향을 고려해야 합니다. 또한, 선택한 색상이 전달하는 감정과 분위기 역시 고려해야 합니다. 각 색상이 전달하는 감정과 분위기를 고려하여 조화로운 조합을 찾는 것이 중요합니다. 또한, 선택한 색상이 전달하는 감정과 분위기 역시 고려해야 합니다. 특정 색에는 진정성, 안정성, 활기, 열정 등의 특성이 있으므로 이러한 색의 감정적인 영향을 고려하여 조합을 선정하는 것이 좋습니다.')
(0.5163911998271942, '인테리어에서 수납 공간을 늘리는 방법은 다양합니다. 예를 들어, 가구를 활용한 수납, 벽면을 활용한 수납장 설치, 다용도 가구의 활용, 수납함과 서랍을 활용하여 숨겨진 수납 공간을 만들거나 옷장 안에 추가 선반을 설치하는 등의 방법을 통해 효과적인 수납 공간을 만들 수 있습니다.')
(0.569209998846054, '인테리어에서 색상 선택 시 고려해야 할 요소는 다양합니다. 먼저, 공간의 크기와 채광 상태를 고려하여 어두운 공간에는 따뜻한 색상을하고, 밝은 공간에는 산뜻한 컬러를 골라야 합니다. 또한, 선택한 색상이 가구와 어우러지는지 확인하여 공간의 조화를 유지하는 것이 중요합니다. 선택한 색상이 전달하는 감정과 분위기 역시 고려해야 합니다. 각 색상은 진정함, 안정성, 활기, 열정과 같은 감정적인 특성을 가지므로 이러한 특성을 고려하여 조합을 만들어야 합니다. 그러니까, 색상 선택 시에

'인테리어에서 색상 선택 시 고려해야 할 요소는 다양합니다. 먼저, 공간의 크기와 채광 상태를 고려하여 어두운 공간에는 따뜻한 색상을하고, 밝은 공간에는 산뜻한 컬러를 골라야 합니다. 또한, 선택한 색상이 가구와 어우러지는지 확인하여 공간의 조화를 유지하는 것이 중요합니다. 선택한 색상이 전달하는 감정과 분위기 역시 고려해야 합니다. 각 색상은 진정함, 안정성, 활기, 열정과 같은 감정적인 특성을 가지므로 이러한 특성을 고려하여 조합을 만들어야 합니다. 그러니까, 색상 선택 시에는 공간 정보, 채광 상황, 가구와의 일관성, 그리고 색상이 전달하는 감정과 분위기를 모두 고려해야 합니다.'

In [149]:
max_len = 200

Chatbot('인테리어 잘하고 싶은데 팁좀', loaded_model_ep5, tokenizer, max_len, device)

question 인테리어 잘하고 싶은데 팁좀
(0.4607445776462555, '인테리어에서 사용되는 바닥재 중 목재 바닥은 자연스러운 질감과 아름다운 색상으로 따뜻하고 편안한 분위기를 조성해주며, 공간에 자연스러운 아름다움을 더해줍니다. 또한, 저자극성의 속성으로 인해 걷기 편안하고 고급스러운 느낌을 줍니다. 물론, 유지보수가 간편하고 내구성이 뛰어나 매년 갱신페인팅이 필요하지 않은 등 경제적인 장점도 있습니다. 따라서 인테리어에 목재 바닥을 사용함으로써 아늑하고 고급스러운 분위기를 조성할 수 있습니다.')
(0.5609440237283706, '인테리어에서 색상 선택은 중요한데, 먼저, 색상의 밝기와 어둠의 정도를 고려하여 조화롭게 조합하는 것이 중요합니다. 예를 들어, 밝은 색과 어두운 색상의 벽에는 중립적인 톤인 검정색상을 선택하거나, 컬러인 청록색상을 고려할 수 있습니다. 또한, 선명하고 명도가 높은 색상을 활용하여 공간의 분위기를 조절할 수 있습니다. 마지막으로, 색상을 선택할 때는 실제로 적용하여 공간에 어떻게 작용하는지 확인해야 합니다.')
(0.5197489142417907, '인테리어에서 수납 공간을 늘리는 방법은 다양합니다. 예를 들어, 가구를 활용한 수납, 벽면을 활용한 수납장 설치, 다용도 가구의 활용, 수납함과 서랍을 활용하여 숨겨진 수납 공간을 만들거나 옷장 안에 추가 선반을 설치하는 등의 방법을 통해 효과적인 수납 공간을 만들 수 있습니다. 또한, 가로수길을 활용한 벽면 수납, 가구 아래 공간을 활용한 수납, 다용도실을 적재적소에 설치하여 공간을 효율적으로 활용하는 방법도 효과적입니다.')
(0.5197489142417907, '인테리어에서 수납 공간을 늘리는 방법은 다양합니다. 예를 들어, 가구를 활용한 수납, 벽면을 활용한 수납장 설치, 다용도 가구의 활용, 수납함과 서랍을 활용하여 숨겨진 수납 공간을 만들거나 옷장 안에 추가 선반을 설치하는 등의 방법을 통해 효과적인 수납 공간을 만들 수 있습니다. 또한, 가로수길을 활용한

'인테리어에서 색상 선택은 중요한데, 먼저, 색상의 밝기와 어둠의 정도를 고려하여 조화롭게 조합하는 것이 중요합니다. 예를 들어, 밝은 색과 어두운 색상의 벽에는 중립적인 톤인 검정색상을 선택하거나, 컬러인 청록색상을 고려할 수 있습니다. 또한, 선명하고 명도가 높은 색상을 활용하여 공간의 분위기를 조절할 수 있습니다. 마지막으로, 색상을 선택할 때는 실제로 적용하여 공간에 어떻게 작용하는지 확인해야 합니다.'

In [150]:
max_len = 200

Chatbot('원목 사이딩과 원목 인테리어의 관계가 뭐야?', loaded_model_ep5, tokenizer, max_len, device)

question 원목 사이딩과 원목 인테리어의 관계가 뭐야?
(0.6405968487262725, '원목사이딩은 주로 목재를 사용해서 외벽을 마감하는 데 사용되는 재료입니다. 이 재료는 자연스러운 우드 그레인과 텍스처를 제공하며, 외부 요인에 강하고 내구성이 뛰어나며, 전통적이고 자연스러운 외관을 유지할 수 있는 장점이 있습니다.')
(0.5955526381731033, '원목사이딩은 주로 재생 용목을 사용하여 만든 외장 벽재로, 재생 용목의 환경 친화적 특성을 갖고 있습니다. 이러한 특성은 건물의 외부에 사용할 때 내구성을 보장하면서도 자연 친화적인 느낌을 제공해줍니다.')
(0.6088692843914032, '원목사이딩의 장점은 주로 단열성능이 우수하며, 외장재 중에서 가장 친환경적인 재료로 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 색감을 살려 건물에 따뜻하고 자연스러운 분위기를 부여할 수 있습니다.')
(0.6088692843914032, '원목사이딩의 장점은 주로 단열성능이 우수하며, 외장재 중에서 가장 친환경적인 재료로 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 색감을 살려 건물에 따뜻하고 자연스러운 분위기를 부여할 수 있습니다.')
(0.6403052896261215, '원목사이딩은 일반적으로 소나무, 잣나무, 전나무와 같은 원목으로 만들어진 사이딩입니다. 이 사이딩은 내구성이 매우 강하며 자연스러운 질감을 갖고 있어 외관적으로 매우 매력적입니다. 또한 내열성과 보온성도 매우 뛰어나다는 특징을 가지고 있습니다.')
원목사이딩은 주로 목재를 사용해서 외벽을 마감하는 데 사용되는 재료입니다. 이 재료는 자연스러운 우드 그레인과 텍스처를 제공하며, 외부 요인에 강하고 내구성이 뛰어나며, 전통적이고 자연스러운 외관을 유지할 수 있는 장점이 있습니다.


'원목사이딩은 주로 목재를 사용해서 외벽을 마감하는 데 사용되는 재료입니다. 이 재료는 자연스러운 우드 그레인과 텍스처를 제공하며, 외부 요인에 강하고 내구성이 뛰어나며, 전통적이고 자연스러운 외관을 유지할 수 있는 장점이 있습니다.'

In [151]:
Chatbot('원목 사이딩과 원목 인테리어의 관계가 뭐야?', loaded_model_ep6, tokenizer, max_len, device)

question 원목 사이딩과 원목 인테리어의 관계가 뭐야?
(0.5974265098571777, '원목사이딩의 장점은 주로 단열성이 우수하고 가장 친환경적인 외장재 중 하나이며 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 따뜻한 분위기를 제공하며, 내구성이 뛰어나서 오랫동안 사용할 수 있는 점도 장점으로 꼽힙니다. 또한 나무가 가지고 있는 자연스러운 아름다움과 고급스러운 느낌을 살릴 수 있어 건물의 외관을 더욱 아름답게 만들어 줍니다.')
(0.5382835000753402, '원목사이딩은 주로 재생 용목으로 만들어져 있습니다. 이는 재생 용목의 환경 친화적인 특성을 가지고 있어 건물의 외부에서 내구성을 유지할 수 있으면서도 자연 친화적인 느낌을 제공합니다.')
(0.6046719998121262, '원목사이딩의 장점은 주로 단열성이 우수하고 가장 친환경적인 외장재 중 하나이며 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 따뜻한 분위기를 제공하며, 내구성이 뛰어나고 자연스러운 아름다움과 고급스러운 느낌을 살릴 수 있다는 점도 장점으로 꼽힙니다. 또한 나무가 가지고 있는 자연스러운 아름다움과 고급스러운 느낌을 살릴 수 있어 건물의 외관을 더욱 아름답게 만들어 줍니다.')
(0.6098816722631454, '원목사이딩의 장점은 주로 단열성이 우수하고 가장 친환경적인 외장재 중 하나이며 시공이 비교적 쉽다는 점입니다. 또한 자연스러운 우드 텍스처와 따뜻한 분위기를 제공하며, 내구성이 뛰어나고 자연스러운 우드 텍스처를 통해 건물에 따뜻하고 자연스러운 분위기를 부여할 수 있습니다.')
(0.6980811595916748, '원목사이딩은 건물의 외벽을 장식하고 보호하는데 사용되는 재료이다. 이 재료는 실제 자연 목재로 만들어진 사이딩을 말합니다. 내구성이 뛰어나고 자연스러운 질감을 가지고 있다. 또한, 자연스럽고 따뜻한 분위기를 연출하기 위해 벽면이나 가구를 선택할 때 좋다.')
원목사이딩은 건물의 외벽을 장식하고 보호하는데 사용

'원목사이딩은 건물의 외벽을 장식하고 보호하는데 사용되는 재료이다. 이 재료는 실제 자연 목재로 만들어진 사이딩을 말합니다. 내구성이 뛰어나고 자연스러운 질감을 가지고 있다. 또한, 자연스럽고 따뜻한 분위기를 연출하기 위해 벽면이나 가구를 선택할 때 좋다.'

In [152]:
reset_seeds(SEED)

max_len = 200
answer_list = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot(i, loaded_model_ep6, tokenizer, max_len, device)
    answer_list.append(answer)

answer_list

  0%|          | 0/130 [00:00<?, ?it/s]

question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')


  1%|          | 1/130 [00:06<14:15,  6.63s/it]

(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
(0.6660684507144125, '도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이러한 현상이 발생하는 이유, 누구의 책임이며 해결 방법에 대해 설명드리겠습니다. 첫째, 습도가 높은 경우 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다. 둘째, 도배지 주변의 누수를 해결하고 곰팡이가 발생한 도배지 부분을 교체하는 것이 필요할 수 있습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.')
(0.6595058374461673, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 

  2%|▏         | 2/130 [00:17<19:05,  8.95s/it]

...

 99%|█████████▉| 129/130 [10:33<00:06,  6.03s/it]

(0.7121744010759437, '벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다.')
벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습

100%|██████████| 130/130 [10:35<00:00,  4.89s/it]

(0.680092841386795, '카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.')
카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.


['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
 '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 이때는 누수 부분을 보수하고, 곰팡이가 발생한 도배지의 부분 또는 전체를 교체하는 것이 필요합니다. 이러한 작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.',
 '큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.',
 '철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다.',
 '도배지 한 롤에 몇평 할 수 있어? 또한, 결로 판정 기준이 어떻게 돼?',
 '철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건축 구조입니다. 이 구조는 철골의 강도와 내구성을 바탕으로 건물을 지지하고, 철근은 인장력을, 콘크리트는 압축력을 담당하여 건물의 강성과 튼튼함을 보장합니다. 이러한 구조는 고층 건물이나 고층 건물에서 안정성을 확보하는

In [153]:
reset_seeds(SEED)

max_len = 200
answer_list_ep5 = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot(i, loaded_model_ep5, tokenizer, max_len, device)
    answer_list_ep5.append(answer)

answer_list_ep5

  0%|          | 0/130 [00:00<?, ?it/s]

question 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7005581558514864, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
(0.7502022729765985, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 데단열 습도조절과 같은 효과가 없는 점이 단점입니다.')


  1%|          | 1/130 [00:04<10:22,  4.83s/it]

(0.6863206061033101, '방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 것은 비싼 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.')
방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 데단열 습도조절과 같은 효과가 없는 점이 단점입니다.
question 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
(0.6736955233158604, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 묻어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 녹을 수 있고, 제습기를 가동하여 적정한 습도를 유지하는 것이 중요합니다. 또한, 도배 후 바닥이나 벽면에 같은 환경에서는 금속재료가 부식되지 않도록 주의해야 합니다.')
(0.6588909838650678, '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지에 녹이 묻어나올 수 있습니다. 따라서 공간 내 습도가 높은 경우에는 제습기를 사용하여 적정 습도를 유지하는 것이 중요합니다. 또한, 도배

  2%|▏         | 2/130 [00:15<17:06,  8.02s/it]

...

 99%|█████████▉| 129/130 [10:37<00:06,  6.09s/it]

(0.7121744010759437, '벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다.')
벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습

100%|██████████| 130/130 [10:40<00:00,  4.92s/it]

(0.680092841386795, '카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.')
카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.


['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 데단열 습도조절과 같은 효과가 없는 점이 단점입니다.',
 '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 묻어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 녹을 수 있고, 제습기를 가동하여 적정한 습도를 유지하는 것이 중요합니다. 또한, 도배 후 바닥이나 벽면에 같은 환경에서는 금속재료가 부식되지 않도록 주의해야 합니다.',
 '큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.',
 '일반적으로, 고층 건물에서는 하중을 지지하기 위해 아래층의 보나 기둥, 보를 사용하는 것이 일반적입니다. 또한 조명을 적절하게 활용하여 공간의 분위기를 조성하고, 단열재의 밀착 시공성을 강화해야 합니다.',
 '도배지의 건조는 일반적으로 약 8시간이 소요됩니다. 이 시간에는 기존 벽지를 제거하고 벽면을 보수하며, 도배 과정이 포함됩니다. 하지만 실제로 소요되는 시간은 도배할 면적의 크기와 추가 작업 여부에 따라 다를 수 있습니다. 따라서 정확한 시공 일정과 비용을 알기 위해서는 시공사와 상담하는 것이 좋습니다. 동일한 내용이지만 다른 방식으로 표현해주었습니다.',
 '철근철골콘크리트 구조는 일반적으로 도심에서 가장 많이 이용하는 구조 중 하나입

In [ ]:
reset_seeds(SEED)

max_len = 200
answer_list_ep4 = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot(i, loaded_model_ep4, tokenizer, max_len, device)
    answer_list_ep4.append(answer)

answer_list_ep4

In [ ]:
reset_seeds(SEED)

max_len = 200
answer_list_ep3 = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot(i, loaded_model_ep3, tokenizer, max_len, device)
    answer_list_ep3.append(answer)

answer_list_ep3

In [ ]:
answer_list_ep5

In [ ]:
for i in range(len(test['질문'].tolist())):
    print(f'Question {i+1}',test['질문'].tolist()[i])

    print('>> Answer_epoch3 :', answer_list_ep5[i])
    print('>> Answer_epoch4 :', answer_list_ep5[i])
    print('>> Answer_epoch5 :', answer_list_ep5[i])
    print('>> Answer_epoch6 :', answer_list[i])
    print()
# %

# 추론 코드 고도화
- len이 너무 짧을 경우, (표준편차 기준?) : 그 다음 유사도가 높은 애를 뽑기
- 너무 짧은거의 기준을? 뭐로 하지

In [158]:
import re

def Chatbot(input_text, model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    text_t = input_text.strip()
    print('question', text_t)
    text = '<q>' + text_t + '</s><a>'
    x = tokenizer.encode(text, return_tensors='pt').to(device)
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    similarity_scores = []

    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    # repetition_penalty=1.0, # Rule-based 방식 사용하려면 off 해야 함
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)
        # result_ids = model.generate(x,max_length=max_len,num_beams=1,num_return_sequences=1,no_repeat_ngram_size=4,use_cache=True,do_sample=True,temperature=0.9,top_k=90,top_p=0.95,early_stopping=True)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        generated_text = generated_text.replace('\n', '')
        generated_text = re.sub(r'\.[^.]*$', '.', generated_text)

        similarity_score = evaluate_similarity(text_t, generated_text)
        generated_texts.append(generated_text)
        similarity_scores.append(similarity_score)

        print(similarity_score[i], generated_texts[i]) # 실제 돌릴땐 off

        len_list = [len(i) for i in generated_texts]
        avg_len = sum(len_list) / len(len_list)
        std_dev = np.std(len_list)

        if similarity_score > best_similarity_score and len(generated_text) < avg_len*0.8:
            best_similarity_score = similarity_score
            best_generated_text = generated_text


        # if similarity_score > best_similarity_score:
        #     best_similarity_score = similarity_score
        #     best_generated_text = generated_text

    print(best_generated_text)


    return best_generated_text

In [167]:
import re
import numpy as np

def preprocess_text(text):
    text = '<q>' + text + '</s><a>'
    return text

def Chatbot_length(input_text, model, tokenizer, max_len, device, num_samples=5):
    model.eval()

    input_text = input_text.strip()
    print('질문:', input_text)

    processed_text = preprocess_text(input_text)
    x = tokenizer.encode(processed_text, return_tensors='pt').to(device)
    q_len = len(processed_text) + 1

    generated_texts = []
    similarity_scores = []

    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    num_return_sequences=1,
                                    do_sample=True,
                                    temperature=0.87,
                                    top_k=27,
                                    top_p=0.7)

        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]

        generated_text = generated_text.replace('\n', '')
        generated_text = re.sub(r'\.[^.]*$', '.', generated_text)

        generated_texts.append(generated_text)
        similarity_score = evaluate_similarity(input_text, generated_text)
        similarity_scores.append(similarity_score)

        print(f"similarity: {similarity_score},  {generated_text}")

    avg_len = np.mean([len(text) for text in generated_texts])

    best_generated_text = None
    best_similarity_score = -1.0

    for text, score in zip(generated_texts, similarity_scores):
        if len(text) > avg_len * 0.4 and text[-1] != '?':  # 평균보다 40% 이상인 경우만 고려
            if score > best_similarity_score:
                best_similarity_score = score
                best_generated_text = text

    print(f"\nbest_generated_text: {best_generated_text}\n")

    return best_generated_text

# 사용 예시
# best_response = Chatbot(input_text, model, tokenizer, max_len, device, num_samples=5)
# print(best_response)


In [168]:
Chatbot_length(test['질문'].tolist()[116], loaded_model_ep6, tokenizer, max_len, device)

질문: 외단열과 내단열의 차이점은 무엇이며, 각각의 시공방법이 어떻게 단열 효과에 영향을 미치나요?
similarity: 0.6582108036109379,  외단열은 건물의 외피와 콘크리트 구조 사이에 단열재를 부착하여, 건물의 겉면을 잘 감싸고 열을 효과적으로 막아줍니다. 이를 통해 열이 탈출하거나 외부에서 냉기가 유입되는 문제로 인한 결로나 곰팡이 등의 손상이 줄어듭니다. 그러나 외단열을 적용하고자 할 때에는 외부 마감재의 선택이 필수적입니다. 또한, 외부 작업은 날씨에 영향을 받기 때문에 너무 더울 때나 습한 날씨에는 단열재 부착이 어려울 수 있거나 탈락할 가능성이 있어 숙련된 시공자의 도움이 필요합니다.
similarity: 0.637338736653328,  외단열을 시공하면 냉·난방비를 절감함으로써 에너지 절약 효과를 얻을 수 있으며, 외부 온도 변화에 따른 실내 환경의 안정성이 향상됩니다. 뿐만 아니라, 외단열은 결로와 곰팡이 등의 발생을 줄여 주어 건강한 실내 환경을 유지할 수 있는 장점이 있습니다.
similarity: 0.6720480609947527,  외단열은 건물의 콘크리트 골조와 외부 마감재 사이에 단열재를 부착하는 방식입니다. 단열재가 건축물 겉면을 감싸주고 있기 때문에 단열 효과가 우수하며, 열이 빠져나가고 냉기가 들어오는 현상으로 생기는 결로와 곰팡이 등의 하자 발생이 적습니다. 다만, 외단열을 희망하는 경우 외부 마감재가 필수적으로 동반되어야 하며, 외부에서 진행되는 작업은 날씨의 영향을 많이 받기 때문에 너무 덥거나 습한 날에는 단열재 부착이 어렵거나 탈락하는 현상이 발생할 수 있어 숙련된 시공자의 작업이 중요합니다.
similarity: 0.6428377801858925,  외단열은 건물의 외피와 콘크리트 구조 사이에 단열재를 부착하여, 건물의 겉면을 잘 감싸고 열을 효과적으로 막아줍니다. 이는 건물 내부의 온도를 일정하게 유지하기 위해 열이 빠져나가거나 냉기가 들어오는 문제를 예방합니다.   그러나 외단열을 적용하고자 할

'외단열은 건물의 콘크리트 골조와 외부 마감재 사이에 단열재를 부착하는 방식입니다. 단열재가 건축물 겉면을 감싸주고 있기 때문에 단열 효과가 우수하며, 열이 빠져나가고 냉기가 들어오는 현상으로 생기는 결로와 곰팡이 등의 하자 발생이 적습니다. 다만, 외단열을 희망하는 경우 외부 마감재가 필수적으로 동반되어야 하며, 외부에서 진행되는 작업은 날씨의 영향을 많이 받기 때문에 너무 덥거나 습한 날에는 단열재 부착이 어렵거나 탈락하는 현상이 발생할 수 있어 숙련된 시공자의 작업이 중요합니다.'

In [169]:
reset_seeds(SEED)

max_len = 200
answer_list_ep6_length = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot_length(i, loaded_model_ep6, tokenizer, max_len, device)
    answer_list_ep6_length.append(answer)

answer_list_ep6_length

  0%|          | 0/130 [00:00<?, ?it/s]

질문: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.


  1%|          | 1/130 [00:05<10:51,  5.05s/it]

similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

best_generated_text: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

질문: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
similarity: 0.6660684507144125,  도배지에 생긴 녹색 또는 갈색의 녹은 녹오염으로 알려져 있습니다. 이러한 현상이 발생하는 이유, 누구의 책임이며 해결 방법에 대해 설명드리겠습니다. 첫째, 습도가 높은 경우 도배지 내부에 있는 금속이 산화되어 녹이 발생할 수 있습니다. 책임소재: 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법: 제습기를 가동하거나 실내 적정 습도를 유지하기 위해 환기를 통해 습기를 제거하는 것이 중요합니다. 전문가의 도움을 받아 복원 작업을 권장합니다. 둘째, 도배지 주변의 누수를 해결하고 곰팡이가 발생한 도배지 부분을 교체하는 것이 필요할 수 있습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.
similarity: 0.6595058374461673,  도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통

  2%|▏         | 2/130 [00:15<17:46,  8.33s/it]

...

 99%|█████████▉| 129/130 [11:51<00:06,  6.10s/it]

similarity: 0.7121744010759437,  벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다.

best_generated_text: 벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습

100%|██████████| 130/130 [11:53<00:00,  5.49s/it]

similarity: 0.680092841386795,  카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.

best_generated_text: 카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.



['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
 '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정하게 유지하기 위해 제습기를 가동하고, 환기를 통해 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 이때는 누수 부분을 보수하고, 곰팡이가 발생한 도배지의 부분 또는 전체를 교체하는 것이 필요합니다. 이러한 작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.',
 '큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.',
 '철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이러한 문제를 해결하기 위한 방법 중 하나는 내화성 구조물을 사용하는 것입니다. 예를 들어, 건물을 내화벽으로 둘러싸거나 내화성 자재인 석고보드를 사용하여 화재로 인한 파손을 최소화할 수 있습니다. 이러한 내화재 보강은 화재 발생 시 안전을 위해 필수적입니다.',
 '도배지의 끝 부분이 들뜨거나 좀 더 꼼꼼한 마무리를 하고 싶을 때 필요합니다.',
 '철근철골콘크리트는 철골과 철근, 그리고 콘크리트를

In [176]:
model_original

'kogpt2_80000_5epoch_0.18720loss'

In [177]:
reset_seeds(SEED)

max_len = 200
answer_list_original = []
for i in tqdm(test['질문'].tolist()) :
    answer = Chatbot_length(i, loaded_model_original, tokenizer, max_len, device)
    answer_list_original.append(answer)

answer_list_original

  0%|          | 0/130 [00:00<?, ?it/s]

질문: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.


  1%|          | 1/130 [00:07<17:02,  7.92s/it]

similarity: 0.7005581558514864,  방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

best_generated_text: 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

질문: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
similarity: 0.6593008004483722,  도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 생기는 것을 말합니다. 이 현상은 일반적으로 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지 안쪽의 금속이 녹오염되어 녹이 번식할 수 있습니다. 따라서 공간 내 습도를 적정 수준으로 유지하고, 필요하다면 제습기나 환풍기를 사용하여 습도를 조절하는 것이 필요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이까지 발생할 수 있습니다. 따라서 누수 부분을 보수하고, 곰팡이가 번식한 도배지 일부 또는 전체를 교체해야 할 수도 있습니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.
similarity: 0.6551192234862934,  해당 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 현상으로, 주로 다음과 같은 원인들이 있을 수 있습니다. 1. 높은 습도로 인한 금속 손상 - 원인: 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어

  2%|▏         | 2/130 [00:18<19:48,  9.28s/it]

...

 99%|█████████▉| 129/130 [11:29<00:06,  6.07s/it]

similarity: 0.7121744010759437,  벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다.

best_generated_text: 벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 건축 물리적 변화 원인 : 건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정이 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 석고수정이 있는 부분을 수정하고 전문가에게 정밀 진단을 받아야 합니다. 2. 공간 내 높은 습도 원인 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 3. 구조적 결함 원인 : 건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습

100%|██████████| 130/130 [11:32<00:00,  5.32s/it]

similarity: 0.680092841386795,  카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.

best_generated_text: 카페트의 기대수명은 6년입니다. 그리고 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 용이하다는 장점이 있습니다.



['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.',
 '도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정 수준으로 유지하고, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이 번식할 수 있으므로 누수 부분을 보수하고, 곰팡이가 번식한 도배지 일부 또는 전체를 교체해야 합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.',
 '큐블럭의 단점은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점입니다. 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.',
 '고층 건물을 지탱하는 여러 가지 방법이 있습니다. 가장 일반적인 방법은 석고나 벽돌 등의 재료와 <unk>글, 콘크리트 등을 사용하여 건물을 지탱하는 것입니다. 이러한 재료들은 건물의 무게를 지탱하지 않고 벽체와 인장력을 고려하여 건축될 수 있도록 도와줍니다. 또한, 철근 콘크리트 구조 또한 존재하기 때문에 바닥과 벽체의 이음부를 보호하고 지지할 수 있습니다. 그리고 콘크리트 표면에 보강재를 밀착하고 접착제를 사용하여 건물을 지탱하는 방법도 있습니다. 이러한 방법들을 조합하여 다양한 건물 구성 과정에서 효과적으로 건물을 지을 수 있습니다.'

In [ ]:
answer_list_ep4

In [ ]:
len(answer_list)

In [170]:
data = {
        'answer_list_original' : answer_list_original,
        'answer_list_ep6': answer_list,
        'answer_list_ep6_length': answer_list_ep6_length,
        # 'answer_list_ep4': answer_list_ep4,
        # 'answer_list_ep3': answer_list_ep3,
        'answer_list_ep5': answer_list_ep5}



In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv(f'{data_path}my_dataframe.csv', index=False)


In [166]:
for i in range(len(test['질문'].tolist())):
    print(f'Question {i+1}',test['질문'].tolist()[i])

    # print('>> Answer_epoch3 :', answer_list_ep5[i])
    # print('>> Answer_epoch4 :', answer_list_ep5[i])
    print('>> Answer_epoch5 :', answer_list_ep5[i])
    print('>> Answer_epoch6 :', answer_list[i])
    print('>> Answer_epoch6 length :', answer_list_ep6_length[i])

    print()

Question 1 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
>> Answer_epoch5 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩을 사용하는 데단열 습도조절과 같은 효과가 없는 점이 단점입니다.
>> Answer_epoch6 : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>> Answer_epoch6 length : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

Question 2 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
>> Answer_epoch5 : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 베어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹이 묻어나올 수 있습니다. 따라서 습도가 높은 환경에서는 도배지 안쪽의 금속이 녹을 수 있고, 제습기를 가동하여 적정한 습도를 유지하는 것이 중요합니다. 또한, 도배 후 바닥이나 벽면에 같은 환경에서는 금속재료가 부식되지 않도록 주의해야 합니다.
>> Answer_epoch6 : 도배

In [179]:
for i in range(len(test['질문'].tolist())):
    print(f'Question {i+1}',test['질문'].tolist()[i])

    # print('>> Answer_epoch3 :', answer_list_ep5[i])
    # print('>> Answer_epoch4 :', answer_list_ep5[i])
    # print('>> Answer_epoch5 :', answer_list_ep5[i])
    # print('>> Answer(Original) :', answer_list_original[i])
    print('>> Answer(Category) :', answer_list_ep6_length[i])

    print()

Question 1 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
>> Answer(Original) : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.
>> Answer(Category) : 방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.

Question 2 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
>> Answer(Original) : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를 적정 수준으로 유지하고, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다. 또한, 도배지 주변의 누수를 조사하고 보수하는 것이 중요합니다.  만일 누수로 인해 도배지가 젖어서 곰팡이가 발생하게 된다면, 녹오염 뿐만 아니라 곰팡이 번식할 수 있으므로 누수 부분을 보수하고, 곰팡이가 번식한 도배지 일부 또는 전체를 교체해야 합니다. 이러한 작업은 전문가의 도움을 받는 것이 좋습니다.
>> Answer(Category) : 도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 일반적으로 고습하거나 누수가 있는 경우에 발생할 수 있습니다.  고습한 환경에서는 

In [180]:
# Embedding Vector 추출
vector_model = SentenceTransformer('distiluse-base-multilingual-cased-v1') # 평가를 위한 Embedding Vector 추출에 활용하는 모델

pred_vector = vector_model.encode(answer_list)
pred_vector.shape

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

(130, 512)

In [181]:
submission = pd.read_csv(f'{data_path}sample_submission.csv')
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TEST_001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TEST_002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TEST_003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TEST_004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,TEST_126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,TEST_127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128,TEST_128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
submission.iloc[:,1:] = pred_vector
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.026558,-0.020150,0.015123,0.017831,0.064715,0.018982,0.002021,-0.006165,-0.006314,...,-0.013723,-0.000593,-0.031992,-0.039757,0.001512,0.022232,0.030489,0.040009,0.001398,0.013257
1,TEST_001,-0.028350,-0.028787,0.020710,0.002237,0.093131,-0.007611,0.011284,-0.008882,-0.014371,...,-0.031483,-0.013664,-0.011553,-0.032545,-0.020437,0.057005,0.001957,-0.016369,0.026494,0.013365
2,TEST_002,0.027461,-0.047922,-0.024391,-0.000196,0.122556,-0.054846,-0.028347,-0.026099,0.043037,...,-0.034606,-0.017103,0.026653,-0.023691,0.022718,0.013030,-0.042122,-0.038602,-0.048148,0.092198
3,TEST_003,-0.006627,0.028780,-0.041766,0.038018,0.047464,-0.053030,-0.024172,-0.001850,-0.024162,...,-0.059843,-0.041408,0.010751,-0.015097,-0.021498,0.066948,-0.013049,-0.000514,-0.026622,0.071312
4,TEST_004,0.006302,-0.055391,0.009241,-0.046285,-0.033827,-0.025063,0.021444,0.025303,-0.063734,...,-0.008971,0.027027,-0.009967,0.000055,-0.048120,-0.021701,-0.001560,-0.047376,0.045128,-0.021770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.029159,-0.044267,0.028296,0.017269,0.118264,-0.067284,0.066199,0.045351,0.050197,...,0.013426,0.009427,0.049926,-0.040957,0.015311,0.051693,0.042989,-0.026956,-0.029771,0.007569
126,TEST_126,0.026621,0.000300,-0.031898,0.040701,0.078308,-0.071102,-0.029177,0.005646,0.015582,...,-0.039710,-0.068038,-0.021990,-0.028564,0.072385,0.030860,-0.010462,-0.042377,0.009644,0.054166
127,TEST_127,-0.017432,0.034415,-0.010878,0.025300,0.130736,0.003531,-0.030830,-0.024952,-0.066184,...,-0.038696,-0.066834,-0.023318,-0.010548,0.024808,0.011876,0.002073,0.024217,-0.013528,0.050744
128,TEST_128,0.036386,-0.027929,-0.063045,0.015607,0.084918,-0.076239,0.011936,0.024018,-0.014002,...,-0.026435,-0.025367,0.003164,-0.060882,-0.018098,0.008412,0.012525,-0.015694,0.026506,0.010083


In [184]:
model_ep6

'category_kogpt2_trial1_epoch6_loss0.1366'

In [186]:
submission.to_csv(f'{data_path}{model_ep6}_length.csv', index=False)
submission.to_csv(f'{model_ep6}_length.csv', index=False)